In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('final.csv')
df.head()

State       Cat brand Region  Value_Sales  \
0          AP  Wellness   Ccn  South  1654638.507   
1   Karnataka  Wellness   Ccn  South  2124709.076   
2  Tamil Nadu  Wellness   Ccn  South  1554919.383   
3          AP  Wellness   Ccn  South  1252527.800   
4   Karnataka  Wellness   Ccn  South  1381235.935   

   Adjusted_SalesVolume_Reported  Promotion_sales  \
0                    160983.9028      137712.5294   
1                    206718.2396      363670.1226   
2                    151281.9808      264323.9658   
3                    121735.1501      227520.5373   
4                    134244.4963      248513.2106   

   Shopper_Activation_contribution  XSR_Incentives_contribution  \
0                                0                            0   
1                                0                            0   
2                                0                            0   
3                                0                            0   
4                                0                            0   

   Consumer_promotions_Spends   ...    Company's Website  Social Network 2  \
0                           0   ...                 19.0              30.0   
1                           0   ...                 28.0              28.0   
2                           0   ...                  5.0              24.0   
3                           0   ...                 26.0              10.0   
4                           0   ...                  1.0              17.0   

   Social Network 1  Organic Searches  Referral  Expert Reviews  \
0              25.0              18.0      29.0            25.0   
1               8.0              10.0      18.0             8.0   
2              13.0               9.0       9.0             9.0   
3              11.0              27.0       2.0            29.0   
4               6.0              17.0      14.0            30.0   

   Aggregators (Price Comparision)  Email  PPC Ad  Forums  
0                             14.0   21.0    25.0    18.0  
1                             14.0    7.0     0.0    19.0  
2                             19.0   37.0     9.0    16.0  
3                             10.0    1.0     4.0     0.0  
4                             34.0   38.0     0.0    23.0  

[5 rows x 55 columns]

In [3]:
# clustering different state in different cluster.
df_AP = df[df['State'] == 'AP']
df_KA = df[df['State'] == 'Karnataka']
df_TM = df[df['State'] == 'Tamil Nadu']

In [4]:
df_AP['Total_Sale'] = df_AP['Adjusted_SalesVolume_Reported']+df_AP['Promotion_sales']
df_KA['Total_Sale'] = df_KA['Adjusted_SalesVolume_Reported']+df_KA['Promotion_sales']
df_TM['Total_Sale'] = df_TM['Adjusted_SalesVolume_Reported']+df_TM['Promotion_sales']

In [5]:
df_AP.drop(columns=['Cat','brand','Region','Value_Sales','Adjusted_SalesVolume_Reported','Promotion_sales',
                    'TVSPENDS','Non-TVGRP (Radio, Newspaper, Magazine, Outdoor etc..)','Competitor price',
                    'Competitor NonTVGRP Spends','Competitor NonTVGRP',
                    'Shopper_Activation_contribution','XSR_Incentives_contribution',
                    'Consumer_promotions_Spends','Final_consu_promotion','QPS_Sel_psku_Spend','Date',
                    'External Factors( Like GDP, Natural calmities etc ,Temperature others)',
                   'Non-TVGRP Spends (Radio, Newspaper, Magazine, Outdoor etc..)','State'],inplace=True)
df_AP.fillna(0,inplace=True)

In [6]:
df_KA.drop(columns=['Cat','brand','Value_Sales','Adjusted_SalesVolume_Reported','Promotion_sales','TVSPENDS',
                    'Non-TVGRP (Radio, Newspaper, Magazine, Outdoor etc..)','Competitor price',
                    'Competitor NonTVGRP Spends','Region','Non-TVGRP Spends (Radio, Newspaper, Magazine, Outdoor etc..)',
                    'Shopper_Activation_contribution','XSR_Incentives_contribution',
                    'Consumer_promotions_Spends','Final_consu_promotion','QPS_Sel_psku_Spend','Date',
                   'Competitor TVGRP Spends',
                    'External Factors( Like GDP, Natural calmities etc ,Temperature others)','State'],inplace=True)

df_KA.fillna(0,inplace=True)

In [7]:
df_TM.drop(columns=['Cat','brand','Region','Value_Sales','Adjusted_SalesVolume_Reported','Promotion_sales',
                    'Shopper_Activation_contribution','XSR_Incentives_contribution','Consumer_promotions_Spends',
                    'Final_consu_promotion','QPS_Sel_psku_Spend','Date',
                    'Non-TVGRP (Radio, Newspaper, Magazine, Outdoor etc..)','Competitor price','Competitor NonTVGRP'
                   ,'Competitor NonTVGRP Spends',
                    'External Factors( Like GDP, Natural calmities etc ,Temperature others)','State'],inplace = True)
df_TM.fillna(0,inplace=True)

In [8]:
def GRPCoff(df,col_no):
    m = len(df)
    lamba = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
    df_GRP = []
    df_final = []
    for j in lamba:
        for i in range(0,m):
            if i == 0:
                df_GRP.append(df.iloc[i,col_no])
            else:
                df_GRP.append(df.iloc[i,col_no] + j * df_AP.iloc[i-1,col_no])
        df_final.append(df_GRP)
        df_GRP = []
    df_finalGRP = pd.DataFrame(df_final).T
    df_Sale = pd.DataFrame(df['Total_Sale'])
    df_Sale.reset_index(inplace=True,drop=True)
    df_feeding = pd.concat([df_finalGRP,df_Sale],axis=1)
    rmse = []
    for i in range(0,len(lamba)):
        LinReg = LinearRegression()
        LinReg.fit(pd.DataFrame(df_finalGRP.iloc[:,i]),df_Sale)
        rmse.append(np.sqrt(mean_squared_error(df_Sale,LinReg.predict(pd.DataFrame(df_finalGRP.iloc[:,i])))))
    final = pd.DataFrame(rmse,columns=['err']).sort_values(by='err',ascending=True).head(1)
    Index_val = final.index.values[0]
    final = []
    rmse= []
    df_final = []
    return lamba[Index_val],df_finalGRP[Index_val]

In [9]:
def GRPFinder(df):
    final_grp_list_no = []
    final_grp_list_name = []
    f = [col for col in df.columns if (('GRP' in col) and ('Spends' not in col))]
    for colno in f:
        if colno in df.columns:
            final_grp_list_no.append(df.columns.get_loc(colno))
            final_grp_list_name.append(colno)
    return final_grp_list_no,final_grp_list_name

In [10]:
def GRPFinder_delete(df):
    final_grp_list_name = []
    f = [col for col in df.columns if (('GRP' in col) and ('new' not in col))]
    return f

In [11]:
def modify_column_name(GList,name):
    mylist = []
    for key,item in enumerate(GList):
        mylist.append(GList[key]+str("_")+str(name))
    return mylist

In [12]:
def combined_together(df):
    df_all = pd.DataFrame()
    GRPListNo,GRPListName = GRPFinder(df)
    for key,grpcol in enumerate(GRPListNo):
        lamba,df_grp_new = GRPCoff(df,grpcol)
        df_all = pd.concat([df_all,df_grp_new],axis=1)
        df.reset_index(inplace=True,drop=True)
    df_all.columns = modify_column_name(GRPListName,lamba)
    df = pd.concat([df,df_all],axis=1)
    return df

In [13]:
df_AP = combined_together(df_AP)
df_KA = combined_together(df_KA)
df_TM = combined_together(df_TM)

In [14]:
# df_AP.drop(columns=GRPFinder_delete(df_AP),inplace=True)
# df_KA.drop(columns=GRPFinder_delete(df_KA),inplace=True)
# df_TM.drop(columns=GRPFinder_delete(df_TM),inplace=True)

In [15]:
# def final_prediction(df,name):
#     y = df['Total_Sale']
#     X = df.drop(columns=['Total_Sale'])
#     finReg = LinearRegression()
#     finReg.fit(X,y)
#     yhat = finReg.predict(X)
#     print(y)
#     print(yhat)
#     SS_Residual = sum((y-yhat)**2)
#     SS_Total = sum((y-np.mean(y))**2)
#     print(SS_Residual)
#     print(SS_Total)
#     r_squared = 1 - (float(SS_Residual))/SS_Total
#     adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-X.shape[1]-1)
#     print(r_squared, adjusted_r_squared)
#     rmse_value = np.sqrt(mean_squared_error(y,yhat))
#     print(rmse_value)
#     alpha = finReg.intercept_
#     beta = pd.DataFrame(finReg.coef_,index=X.columns,columns=[name])
#     return beta,alpha

In [16]:
#AP,alpha1 = final_prediction(df_AP,"AP")
# KA,alpha2 = final_prediction(df_KA,"KA")
# TM,alpha3 = final_prediction(df_TM,"TM")
# final_beta = pd.concat([AP,KA,TM],axis=1)

In [17]:
# final_beta = pd.concat([AP,KA,TM],axis=1)
# final_intercept = pd.DataFrame([alpha1,alpha2,alpha3]).T
# final_intercept.columns = final_beta.columns
# final_intercept.index = ['alpha']
# final_frame = pd.concat([final_beta,final_intercept],axis=0)
# final_frame.to_excel('final_data.xlsx')

In [18]:
df_KA.columns

Index(['Trade_Schemes_Sel_psku_Spend', 'MRP_Price', 'TVGRP', 'Sales_VOL_Tn',
       'Sales_Val_Mn', 'MS_Val', 'MS_Vol', 'NumD_max', 'WtdD_Val_Max',
       'Competitor_TVGRP', 'Competitor TVGRP', 'Competitor NonTVGRP',
       'sku1_promo', 'sku2_promo', 'video_1_views', 'video_1_likes',
       'video_1_dislikes', 'video_1_comment_total', 'video_2_views',
       'video_2_likes', 'video_2_dislikes', 'video_2_comment_total',
       'Average_Temperature', 'Number_Of_Holidays', 'Total_Flights',
       'Company's Website', 'Social Network 2', 'Social Network 1',
       'Organic Searches', 'Referral', 'Expert Reviews',
       'Aggregators (Price Comparision)', 'Email', 'PPC Ad', 'Forums',
       'Total_Sale', 'TVGRP_0.1', 'Competitor_TVGRP_0.1',
       'Competitor TVGRP_0.1', 'Competitor NonTVGRP_0.1'],
      dtype='object')

In [19]:
df_AP_trail = df_AP[['Trade_Schemes_Sel_psku_Spend','sku1_promo','video_1_views','Number_Of_Holidays', 
                     'Organic Searches','Email','Total_Sale', 'TVGRP_0.1', 'Competitor_TVGRP_0.1']]

df_KA_trail = df_KA[['Trade_Schemes_Sel_psku_Spend','sku1_promo','video_1_views','Number_Of_Holidays', 
                     'Organic Searches','Email','Total_Sale', 'TVGRP_0.1','Competitor_TVGRP_0.1']]

df_TM_trail = df_TM[['Trade_Schemes_Sel_psku_Spend','sku1_promo','video_1_views','Number_Of_Holidays', 
                     'Organic Searches','Email','Total_Sale', 'TVGRP_0.1', 'Competitor_TVGRP_0.1']]

In [20]:
def final_output_func(df):
    y = pd.DataFrame(df['Total_Sale'])
    X = df.drop(columns=['Total_Sale'])
    X = sm.add_constant(X)
    model = sm.OLS(y,X).fit()
    return model.rsquared,model.rsquared_adj,model.params,model.predict(X)

In [21]:
Rsquare_AP,Adj_Rsquare_AP,Summ_AP,Predicted_AP= final_output_func(df_AP_trail)
Rsquare_KA,Adj_Rsquare_KA,Summ_KA,Predicted_KA= final_output_func(df_KA_trail)
Rsquare_TM,Adj_Rsquare_TM,Summ_TM,Predicted_TM= final_output_func(df_TM_trail)

In [22]:
df_AP_trail = pd.concat([df_AP_trail,Predicted_AP],axis=1)
df_KA_trail = pd.concat([df_KA_trail,Predicted_KA],axis=1)
df_TM_trail = pd.concat([df_TM_trail,Predicted_TM],axis=1)

In [23]:
final_output_df = pd.concat([Summ_AP,Summ_KA,Summ_TM],axis=1)
final_output_df.columns = ['AP','KA','TM']
final_output_df
# final_Rsquare = pd.DataFrame([Rsquare_AP,Rsquare_KA,Rsquare_TM],columns=['AP','KA','TM'],axis=0)
# final_AdjRsquare = pd.DataFrame([Adj_Rsquare_AP,Adj_Rsquare_KA,Adj_Rsquare_TM],columns=['AP','KA','TM'],axis=0)

AP            KA             TM
const                         201053.268205   5426.255132  142778.288978
Trade_Schemes_Sel_psku_Spend  100124.135571  61426.293472   79735.194822
sku1_promo                       -15.764284      5.750460       0.818564
video_1_views                      0.062226      0.003948      -0.021586
Number_Of_Holidays            -12113.124154   6454.963907  -27963.228396
Organic Searches                1667.778343  -1086.964887   -2941.050301
Email                          -3883.245003  -1587.418667    -357.275020
TVGRP_0.1                        -91.596726    302.516540    -139.665309
Competitor_TVGRP_0.1              48.918390    -78.767137      11.612783

In [24]:
df_AP_trail.to_excel('APdata.xlsx')
df_KA_trail.to_excel('KAdata.xlsx')
df_TM_trail.to_excel('TMdata.xlsx')

In [25]:
final_output_df.to_excel('final_output.xlsx')

In [26]:
print("Rsquare_AP -",Rsquare_AP)
print("Rsquare_KA -",Rsquare_KA)
print("Rsquare_TM -",Rsquare_TM)
print("Adj_Rsquare_AP -",Adj_Rsquare_AP)
print("Adj_Rsquare_KA -",Adj_Rsquare_KA)
print("Adj_Rsquare_TM -",Adj_Rsquare_TM)

Rsquare_AP - 0.9078910001093253
Rsquare_KA - 0.8307855217025575
Rsquare_TM - 0.9028870050384459
Adj_Rsquare_AP - 0.8157820002186507
Adj_Rsquare_KA - 0.661571043405115
Adj_Rsquare_TM - 0.7919007250823841
